### Fill In Missing 

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pathlib
import time
import pyspark.pandas as ps

import numpy as np

from datetime import date, datetime, timedelta
from pyspark.pandas.indexes.datetimes import DatetimeIndex
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col,isnan,when,count, when, col, rank, lit, monotonically_increasing_id, date_trunc, udf, min, max, explode
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, DateType, StringType

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/17 12:48:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
glucose_data_schema=StructType([StructField('NumId', IntegerType(), True),
                                        StructField('PatientId', StringType(), True),
                                        StructField('Value', FloatType(), True),
                                        StructField('GlucoseDisplayTime', TimestampType(), True),
                                        StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                        StructField('GlucoseDisplayDate', DateType(), True)])

In [8]:
read_in_types = ['training_set', 'test_set', 'val_set']
data_types = ['train', 'test', 'val']

# for data_type in data_types: 
for j in range(0,3): 
    data_location = "/cephfs/train_test_val/" + read_in_types[j]
    allPaths = [str(x) for x in list(pathlib.Path(data_location).glob('*.parquet')) if 'part-00' in str(x)]
    path_counter = 0
    
    for path in allPaths:
        gluc = pd.read_parquet(path, columns=['NumId','GlucoseDisplayTime', 'Value'])
        gluc['GlucoseDisplayTime'] = gluc['GlucoseDisplayTime'].dt.floor('Min')
        gluc = gluc.sort_values(by=['NumId', 'GlucoseDisplayTime'])

        min_max = gluc.groupby('NumId')\
                        .agg({'GlucoseDisplayTime' : ['min','max']})

        merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
        starttime = time.time()
        last_idx = len(min_max)-1

        index_counter = 0
        for idx, row in min_max.iterrows():
            #grab all potential dates in range

            min_val = row['GlucoseDisplayTime']['min']
            max_val = row['GlucoseDisplayTime']['max']

            date_df = pd.DataFrame(pd.date_range(min_val, max_val, freq='5min'),\
                                   columns=['GlucoseDisplayTime'])  

            # merge dates with big pypsark df
            id_df = gluc[gluc['NumId'] == idx]

            mean = id_df.Value.mean().round()

            id_df.set_index('GlucoseDisplayTime', inplace=True)    

            date_df.set_index('GlucoseDisplayTime', inplace=True)

            merged = id_df.join(date_df, how='outer',\
                                on='GlucoseDisplayTime', sort=True)

            merged['IsFilledIn'] = 0
            merged.loc[merged.Value.isna(), 'IsFilledIn'] = 1        
            merged.loc[merged.Value.isna(), 'Value'] = mean

            merged['NumId'] = idx

            merged.reset_index(inplace=True)

            merged = merged.drop(columns=['index'])

            merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[0],\
                                                np.array(merged['GlucoseDisplayTime'].iloc[:-1].values)), axis=None)\
                                .astype('datetime64[ns]')

            merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

            len_merged = len(merged)

            # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
            # dont include last row to delete
            indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged & x != 0]

            if len(indexes_to_remove) > 0:
                merged = merged.drop(indexes_to_remove)

            # its ready freddy for some interpoletty
            # merged DF is the dataframe ready to go into interpolation function

            # fill with mean

            merged = merged.drop(columns=['TimeLag', 'Diff'])

            if ((index_counter % 25 != 0) and index_counter != last_idx) or (index_counter == 0):
                merge_df = pd.concat([merge_df, merged])
            elif (index_counter % 25 == 0) or (index_counter == last_idx):
                merge_df = merge_df.astype({'GlucoseDisplayTime': 'datetime64[ns]'})
                
                merge_df.to_parquet('/cephfs/interpolation/' + data_types[j] + '/parquet_' + str(path_counter) + '_' + str(index_counter) + '.parquet')
                merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
               
            
            index_counter += 1

        path_counter += 1
    
    print(data_types[j] + ' done')
        

train done
test done
val done


testing if it worked

In [9]:
train_sample = spark.read.format('parquet').load('/cephfs/interpolation/train/parquet_0_25.parquet')

In [10]:
train_sample.sort('NumId', 'GlucoseDisplayTime').show(20)

+-------------------+-----+-----+----------+-----------------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|__index_level_0__|
+-------------------+-----+-----+----------+-----------------+
|2022-01-31 17:14:00|   63|177.0|       0.0|                0|
|2022-01-31 17:19:00|   63|168.0|       0.0|                1|
|2022-01-31 17:24:00|   63|163.0|       0.0|                2|
|2022-01-31 17:29:00|   63|166.0|       0.0|                3|
|2022-01-31 17:34:00|   63|166.0|       0.0|                4|
|2022-01-31 17:39:00|   63|165.0|       0.0|                5|
|2022-01-31 17:44:00|   63|165.0|       0.0|                6|
|2022-01-31 17:49:00|   63|165.0|       0.0|                7|
|2022-01-31 17:54:00|   63|165.0|       0.0|                8|
|2022-01-31 17:59:00|   63|170.0|       0.0|                9|
|2022-01-31 18:04:00|   63|175.0|       0.0|               10|
|2022-01-31 18:09:00|   63|179.0|       0.0|               11|
|2022-01-31 18:14:00|   63|182.0|       0.0|           

In [11]:
test_sample = spark.read.format('parquet').load('/cephfs/interpolation/test/parquet_0_25.parquet')

In [12]:
test_sample.sort('NumId', 'GlucoseDisplayTime').show(20)

+-------------------+-----+-----+----------+-----------------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|__index_level_0__|
+-------------------+-----+-----+----------+-----------------+
|2022-11-08 17:56:00|  103|203.0|       0.0|                0|
|2022-11-08 18:01:00|  103|210.0|       0.0|                1|
|2022-11-08 18:06:00|  103|219.0|       0.0|                2|
|2022-11-08 18:11:00|  103|229.0|       0.0|                3|
|2022-11-08 18:16:00|  103|233.0|       0.0|                4|
|2022-11-08 18:21:00|  103|236.0|       0.0|                5|
|2022-11-08 18:26:00|  103|237.0|       0.0|                6|
|2022-11-08 18:31:00|  103|234.0|       0.0|                7|
|2022-11-08 18:36:00|  103|226.0|       0.0|                8|
|2022-11-08 18:41:00|  103|219.0|       0.0|                9|
|2022-11-08 18:46:00|  103|214.0|       0.0|               10|
|2022-11-08 18:51:00|  103|211.0|       0.0|               11|
|2022-11-08 18:56:00|  103|198.0|       0.0|           

In [14]:
val_sample = spark.read.format('parquet').load('/cephfs/interpolation/val/parquet_0_25.parquet')

In [15]:
val_sample.sort('NumId', 'GlucoseDisplayTime').show(20)

+-------------------+-----+-----+----------+-----------------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|__index_level_0__|
+-------------------+-----+-----+----------+-----------------+
|2022-08-24 10:39:00|  323| 93.0|       0.0|                0|
|2022-08-24 10:44:00|  323| 99.0|       0.0|                1|
|2022-08-24 10:49:00|  323|100.0|       0.0|                2|
|2022-08-24 10:54:00|  323| 96.0|       0.0|                3|
|2022-08-24 10:59:00|  323| 93.0|       0.0|                4|
|2022-08-24 11:04:00|  323| 94.0|       0.0|                5|
|2022-08-24 11:09:00|  323| 99.0|       0.0|                6|
|2022-08-24 11:14:00|  323|102.0|       0.0|                7|
|2022-08-24 11:19:00|  323|113.0|       0.0|                8|
|2022-08-24 11:24:00|  323|111.0|       0.0|                9|
|2022-08-24 11:29:00|  323|109.0|       0.0|               10|
|2022-08-24 11:34:00|  323|102.0|       0.0|               11|
|2022-08-24 11:39:00|  323|102.0|       0.0|           